<a href="https://colab.research.google.com/github/asrithap07/AITaylorSwiftLyricGenerator/blob/main/Taylor_Swift_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Generation using GPT (Using Huggingface)

## Project Setup

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip uninstall transformers
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.8 MB/s eta 0:00:00


In [ ]:
import torch
import shutil
from torch.utils.data import Dataset, random_split
from transformers import Trainer, TrainingArguments, GPTNeoForCausalLM, GPT2Tokenizer


from google.colab import drive


## Data Preparation

In [ ]:
# Connects colab to google drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# Read the text file and returns list of lines in text
def read_file(file_path):
    with open(file_path) as f:
        lines = [line for line in f]
        # lines.remove("")
    return lines

In [ ]:
def read_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    divided_lines = []
    for i in range(0, len(lines), 7):
        paragraph = ' '.join(lines[i:i+7]).strip()
        divided_lines.append(paragraph)

    result = '\n\n'.join(divided_lines)

    return result

In [ ]:

file_path = "drive/MyDrive/AICamp/nlp/taylorswift/tslyrics2.txt"

def read_file(file_path):
  with open(file_path) as f:
        lines = [line for line in f]
        # lines.remove("")
        return lines

texts = read_file(file_path)
new_text = []
sonnets = []
sonnet = []
lineCount = 0
for text in texts:
  if lineCount < 4:
    lineCount+= 1
    sonnet.append(text)
  else:
    sonnets.append(''.join(sonnet))
    lineCount= 0
    sonnet = []

print(sonnets)

with open(r'drive/MyDrive/AICamp/nlp/taylorswift/tslyrics2.txt', 'w') as fp:
    for item in sonnets:
        # write each item on a new line
        fp.write("%s" % item)

# Remove unnecessary texts


["When the dinner is cold and the chatter gets old\nYou ask for the tab\nOr that moment again, he's insisting that friends\nLook at each other like that\n", "Sometimes giving up is the strong thing\nBegging 'til my knees bled\nBut I've got me\nThat will find you the right thing\n", "When it's time to go\nA perfectly good heart?\nWhy would you wanna break\nWhy would you wanna break\n", 'Why would you wanna break\n  \nThe very first scar?\nWhy would you wanna break\n', "And you've been getting closer and closer\n \nSo don't look at me, you got a girl at home\nDon't look at me, you got a girl at home\n", "If I was a stupid girl\nDon't look at me, you got a girl at home\nAnd everybody knows that, everybody knows that\nCall a cab, lose my number\n", 'And everybody knows that, everybody knows that\n\nAnd darling, it was good\nNever looking down\n', "And the story's got dust on every page\nBut I don't wanna dance\nLike you were in this room\nNever looking down\n", "I bet this time of night, y

In [ ]:
# Prepare sonnets
datas = sonnets[2:-1]
print(len(datas))
for data in datas:
  if len(data)<1:
    datas.remove(data)
print(len(datas))

413
413


In [ ]:
# Custome dataset class to load dataset;
class TaylorSwiftDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            # Encode the descriptions using the GPT-Neo tokenizer
            encodings_dict = tokenizer('<|startoftext|>'
                                        + txt +
                                        '<|endoftext|>',
                                        truncation=True,
                                        max_length=max_length,
                                            padding="max_length")
            input_ids = torch.tensor(encodings_dict['input_ids'])
            self.input_ids.append(input_ids)
            mask = torch.tensor(encodings_dict['attention_mask'])
            self.attn_masks.append(mask)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

## Initialize tokenizer, model

In [ ]:
# Set the random seed to a fixed value to get reproducible results
torch.manual_seed(42)

# Download the pre-trained GPT-Neo model's tokenizer
# Add the custom tokens denoting the beginning and the end
# of the sequence and a special token for padding
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M",
                            bos_token='<|startoftext|>',
                            eos_token='<|endoftext|>',
                            pad_token='<|pad|>')

# Download the pre-trained GPT-Neo model and transfer it to the GPU
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M").cuda()

# Resize the token embeddings because we've just added 3 new tokens
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50259, 768)

## Train/Test Split data

In [ ]:
max_length = max([len(tokenizer.encode(sonnet)) for sonnet in datas])

# Load dataset
dataset = TaylorSwiftDataset(sonnets, tokenizer, max_length)

# Split data into train/val
train_size = int(0.9 * len(dataset))

train_data, val_data = random_split(dataset, [train_size, len(dataset) - train_size])

max_length

61

In [ ]:
tokenizer.batch_decode(val_data[0])

["<|startoftext|> And lead me up the staircase, won't you whisper soft and slow?\n'Cause I see sparks fly whenever you smile\nAnd the sparks fly\n\n<|endoftext|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|>",
 '"""""""""""""""""""""""""""""""""!!!!!!!!!!!!!!!!!!!!!!!!!!!!']

## Train Model

In [ ]:
##Here I will pass the output directory where
#the model predictions and checkpoints will be stored,
###batch sizes for the training and validation steps,
#and warmup_steps to gradually increase the learning rate
learning_rates = [5e-5, 3e-5, 1e-5]

for learning_rate in learning_rates:

     training_args = TrainingArguments(output_dir=f'./results_{learning_rate}',
                                       num_train_epochs=5,
                                       logging_steps=1000,
                                       save_steps=1000,
                                       evaluation_strategy='steps',
                                       eval_steps=1000,
                                       per_device_train_batch_size=2,
                                       per_device_eval_batch_size=2,
                                       warmup_steps=100,
                                       learning_rate=learning_rate,
                                       weight_decay=0.01,
                                       logging_dir=f'./logs_{learning_rate}')

     trainer = Trainer(model=model, args=training_args,
                       train_dataset=train_data,
                       eval_dataset=val_data,
                       # This custom collate function is necessary
                       # to built batches of data
                       data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                   'attention_mask': torch.stack([f[1] for f in data]),
                   'labels': torch.stack([f[0] for f in data])})

#     # Start training process!
     print(f"Training result for learning rate: {learning_rate}")
     trainer.train()
     print("\n\n")

Training result for learning rate: 5e-05


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss





Training result for learning rate: 3e-05


Step,Training Loss,Validation Loss





Training result for learning rate: 1e-05


Step,Training Loss,Validation Loss


BAsed on the results above, it looks like model trained with learning rate = 5e-5 is more promising than others.

In [ ]:
training_args = TrainingArguments(output_dir=f'./results',
                                      num_train_epochs=5,
                                      logging_steps=1000,
                                      save_steps=5000,
                                      evaluation_strategy='steps',
                                      eval_steps=1000,
                                      per_device_train_batch_size=2,
                                      per_device_eval_batch_size=2,
                                      warmup_steps=100,
                                      learning_rate=5e-5,
                                      weight_decay=0.01,
                                      logging_dir=f'./logs')

trainer = Trainer(model=model, args=training_args,
                  train_dataset=train_data,
                  eval_dataset=val_data,
                  # This custom collate function is necessary
                  # to built batches of data
                  data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
              'attention_mask': torch.stack([f[1] for f in data]),
              'labels': torch.stack([f[0] for f in data])})

# Start training process!
trainer.train()


Step,Training Loss,Validation Loss


TrainOutput(global_step=935, training_loss=0.17888679708389038, metrics={'train_runtime': 91.1417, 'train_samples_per_second': 20.518, 'train_steps_per_second': 10.259, 'total_flos': 58195104675840.0, 'train_loss': 0.17888679708389038, 'epoch': 5.0})

In [ ]:
# Save model in the specified file path
trainer.save_model("drive/MyDrive/AICamp/nlp/models/newtestingParameters")

In [ ]:
tokenizer.save_pretrained("drive/MyDrive/AICamp/nlp/models/newtestingParameters")

('drive/MyDrive/AICamp/nlp/models/newtestingParameters/tokenizer_config.json',
 'drive/MyDrive/AICamp/nlp/models/newtestingParameters/special_tokens_map.json',
 'drive/MyDrive/AICamp/nlp/models/newtestingParameters/vocab.json',
 'drive/MyDrive/AICamp/nlp/models/newtestingParameters/merges.txt',
 'drive/MyDrive/AICamp/nlp/models/newtestingParameters/added_tokens.json')

## Checking Model Output

In [ ]:
#generated = tokenizer("<|startoftext|>", return_tensors="pt").input_ids.cuda()
#sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                                # bos_token='<|startoftext|>',
                                # eos_token='<|endoftext|>', pad_token='<|pad|>',
                                #max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=20)
#for i, sample_output in enumerate(sample_outputs):
    #print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

prompt = "Hello"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

output = model.generate(input_ids.cuda(), do_sample=True, top_k=50,
                        max_length=500, min_length=250, repetition_penalty= 5.00, top_p=0.75, temperature=0.6, num_return_sequences=20)


print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


HelloI'm not a bad girl, I just like to hide
Your face in my hair, makin' up for this moment


(Unless you're spelling it that way)
So what am I defending?You call me out on your contrarian shit, but hey (Oh yeah, oh), let's go fight this thing for the years we've been gone
Cause stay away from her, she should know better than thatyyy is wrong and all
And don't say yes, baby! Stayaway from each other, they'll never be friends
Ooh-ah-uh, oooh-oo
(Hey!)


[Chorus: Taylor Swift]
regoroola tropes pressuring SEA masculriageantryahiconomiusesaiideenurrenciesomination},"reditcreationellationdigyillacIALcakesriblyfireennialameronasuryclavetainingayneekaellectpless.pha impover,
ornedxia defund depreciationphilisPhelleelligentriganrarilypeathesive insurgzaibecuehoeitemscontinentaladataiphanyulasleanorualitycientshttrylic,в epidе


In [ ]:
# ! transformers-cli env

## Upload model to huggingface

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

In [ ]:
# Create your repo first to upload the model
api.create_repo(repo_id="gpt2-taylorswift-generator")

HfHubHTTPError: ignored

In [ ]:
# Upload your model to huggingface. You can clone the repo anytime to use the model.
import os

model_pth = "drive/MyDrive/AICamp/nlp/models/newtestingParameters"

files = os.listdir(model_pth)
for fi in files:
    print(os.path.join(model_pth, fi))

    api.upload_file(
        path_or_fileobj=os.path.join(model_pth, fi),
        path_in_repo=fi,
        repo_id="asrithap/gpt2-taylorswift-generator",
        repo_type="model",
    )

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/3.90k [00:00<?, ?B/s]